In [8]:
import cv2
import numpy as np
import random


cap=cv2.VideoCapture(0)                #Captura la cámara web

kernel= np.ones((15,15), np.uint8)     

i=40                                           #
score= 0                              #Declaración de variables usadas en el código.
score_auxiliar=0                               #
lost=0                                         #
misses=0                                       #


random_spinach= random.randint(1,300)      #Posición aleatoria para la caída de la espinaca
random_burger=random.randint(301,590)      #Posición aleatoria para la caída de la hamburguesa
init=cv2.imread('fondogrande.png')               #Lectura imagen de fondo para el inicio del juego
cv2.putText(init, 'Recoge las espinacas', (700,50), cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0),2)
cv2.putText(init, 'Esquiva las hamburguesas', (670,100), cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0),2)
cv2.putText(init, 'Presiona cualquier tecla', (700,200), cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0),2)
cv2.putText(init, 'para iniciar', (700,240), cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0),2)
cv2.putText(init, 'Suerte!!!', (700,350), cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0),2)
cv2.imshow('RECOGE LA ESPINACA', init)    #Muestra la ventana de inicio con la imagen
cv2.waitKey(0)                             #Espera hasta la interacción del usuario por medio de una tecla
cv2.destroyAllWindows()                    #Cierra la ventana


while True:
    
    
    background=cv2.imread('fondonegro.png')          #Lee la imagen del fondo para el juego 
    background= cv2.resize(background,(640,520), interpolation=cv2.INTER_CUBIC) #Redimensiona el tamaño de la ventana para que quede igual que la captura de la cámara
   
    character= cv2.imread('popeye.png')    #Lee la imagen del personaje
    spinach= cv2.imread('espinaca.png')         #Lee la imagen de la espinaca
    burger= cv2.imread('hamburguesa.jpeg')       #Lee la imagen de la hamburguesa
    fil,col,capa= character.shape               #Toma las dimensiones de fila y columna del personaje
    fil_o,col_o,capa=spinach.shape              #Toma las dimensiones de fila y columna de la espinaca
    fil_f, col_f,capa= background.shape         #Toma las dimensiones de fila y columna del fondo
    fil_h,col_h,capa=burger.shape               #Toma las dimensiones de fila y columna de la hamburguesa
     

    
    check, frame= cap.read()                    #Lee lo que hay en cap, que es la captura del video de la cámara web 
    frame= cv2.flip(frame,1)                    #Crea un efecto espejo
    a= frame[:,:,1]                             #Se extrae la componente azul de la imagen
    a= cv2.subtract(a,cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY )) #Se resta la componente azul con la imagen del frame pero en escala de grises
    
    
    ret,a= cv2.threshold(a,30,255,cv2.THRESH_BINARY) #Con la función Threshold se binariza para que solo se vean los objetos azules
    a=cv2.blur(a,(2,2))                           #Se difumina la imagen para que se tome el objeto azul con más claridad
    a=cv2.dilate(a,kernel,iterations=1)           #Se realiza una dilatación para que la imagen se vea mas grande y definida
    a=cv2.Canny(a,100,150)                        #Se obtiene el contorno del objeto azul en pantalla
  
    borders, cnts= cv2.findContours(a.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #Se toma la posición del objeto en la pantalla y se guarda en una matriz de matrices.
    
    if len(borders)!=0:                         #Verifica que haya un objeto con las características definidas en pantalla
        if borders[0][0][0][0]+col<=640:        #Se hace esta verificación para que no se desborde la figura del personaje
            background[(fil_f- fil):fil_f, borders[0][0][0][0]:borders[0][0][0][0]+col] = character #Se muestra el personaje en el fondo de 
            background[i:i+fil_o, random_spinach:random_spinach+col_o, :] = spinach                 #Coloca las espinacas en el fondo
            background[i:i+fil_h, random_burger:random_burger+col_h, :] = burger                    #Coloca las hamburguesas en el fondo
            if i>=370 and i<400 and random_spinach>= borders[0][0][0][0] and random_spinach<= borders[0][0][0][0]+col and random_spinach!=random_burger: #Condiciones de colisión con las espinacas
                
                score=score+1                 #Aumenta el puntaje cuando recoge la espinaca
                score_auxiliar= score         #Se le asigna a score auxiliar lo que tiene score para luego reiniciar la posición de la espinaca
                if score_auxiliar>0:          #Se verifica la condición de que se haya hecho un punto
                    background[(fil_f- fil):fil_f, borders[0][0][0][0]:borders[0][0][0][0]+col] = character #Se cambia la imagen del fondo por el personaje en las dimensiones indicadas
                    i=40                     #Variable i con el valor de 40 para que apartir de este valor caigan los objetos
                    random_spinach=random.randint(1,300)             #Posición aleatoria para la caída de la espinaca
                    score_aux=0                                      #Se reinicia score auxiliar 
           
            
            if i>=370 and i<400 and random_burger>= borders[0][0][0][0] and random_burger<= borders[0][0][0][0]+col and random_spinach!=random_burger: #Condiciones de colisión con la hamburguesa
                
                score=score-1              #Se resta un punto al score por chocarse con la hamburguesa
                misses=misses+1            #Se aumenta 1 a las hamburguesas recogidas
                misses_aux=misses          #Se usa una variable auxiliar para reiniciar la posición de las hamburguesas
                if misses_aux>0:           #Se verifica que haya recogido al menos una hamburguesa
                    background[(fil_f- fil):fil_f, borders[0][0][0][0]:borders[0][0][0][0]+col] = character  #Se pone en la imagen del fondo el personaje en las coordenadas especificadas
                    i=40                   # Variable i en 40 para que desde ese punto caigan los objetos
                    random_burger=random.randint(297,590)  # Posición aleatoria para la caida de las hamburguesas
                    misses_aux=0           #Se reinicia la variable misses_aux
                    
                    
            cv2.putText(background, 'Ganadas:' + str(score), (250,50), cv2.FONT_HERSHEY_DUPLEX,1,(255,255,255),2) #Se ponen en pantalla las espinacas atrapadas
            cv2.putText(background, 'Perdidas:' + str(lost), (250,80), cv2.FONT_HERSHEY_DUPLEX,1,(255,255,255),2)  #Se ponen en pantalla las espinacas que se han perdido
                             
        
        if score==5 or lost==5:               #Condición de que gana o pierde
            break
        cv2.imshow('Recoge la espinaca', background)
        cv2.imshow('',frame)
        key= cv2.waitKey(1)

        if key==ord('q') or key==ord('Q'):     #Verifica si se preiona la letra q o Q para cerrar el programa
            break
        i=i+10                                 #Aumento de la variable i para la caida de los objetos en este caso de a 10 pixeles

        if i>=(fil_f-fil_o):                   #Condición para verificar si la espinaca llegó al piso y se perdió
            i=40                               #Variable i en 40 para que desde ese punto caigan los objetos
            random_spinach=random.randint(1, col_f- col_o) #Posición aleatoria para la caída de la espinaca
            random_burger=random.randint(1, col_f-col_h)   #Posición aleatoria para la caída de la hamburguesa
            lost=lost+1                                 #Aumento de las espinacas perdidas en 1
        
cap.release()
cv2.destroyAllWindows()

background= cv2.imread('fondo.png')                    #lee las imágenes para el fondo, para la pantalla de ganar y la de perder.
win= cv2.imread('olivia.jpg')
game_over= cv2.imread('brutus.jpg')
 
if score== 5:                              #Si el score es igual a 5 quiere decir que ganó, ahi lee la imagen de partida ganada y la muestra
    cv2.imshow('Ganaste', win)
    win= cv2.resize(win,(640,383), interpolation=cv2.INTER_CUBIC)
    cv2.putText(win,'Ganaste!!!!!',(250,50), cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0),3)
    
     
elif lost==5:                              #Si el score es igual a 5 quiere decir que ganó, ahi lee la imagen de partida ganada y la muestra
    cv2.imshow('Perdiste', game_over)
    game_over=cv2.resize(game_over,(408,505), interpolation=cv2.INTER_CUBIC)
    
   
    
    

cv2.waitKey(0)
cv2.destroyAllWindows()
    
    